<a href="https://colab.research.google.com/github/hasanzeynal/Streamlit-Car-Price/blob/main/Modeling/car_price_prediction_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [243]:
#import libraries
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_absolute_error,r2_score,mean_absolute_percentage_error,mean_squared_error
from sklearn.preprocessing import RobustScaler
#ignore all warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
#connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Understanding

In [248]:
#train and test dataframe
train_df = pd.read_csv('/content/drive/MyDrive/datasets/Car_price/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/datasets/Car_price/test.csv')

In [249]:
#looking for a df's that has null value or not
test_df.isna().any().sum(),train_df.isna().any().sum()

(1, 0)

In [250]:
test_df.isnull().sum(),test_df.shape

(ID                     0
 Levy                   0
 Manufacturer           0
 Model                  0
 Prod. year             0
 Category               0
 Leather interior       0
 Fuel type              0
 Engine volume          0
 Mileage                0
 Cylinders              0
 Gear box type          0
 Drive wheels           0
 Doors                  0
 Wheel                  0
 Color                  0
 Airbags                0
 Price               8245
 dtype: int64,
 (8245, 18))

In [251]:
train_df.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


In [252]:
#checking for null value
train_df.isnull().sum()

ID                  0
Price               0
Levy                0
Manufacturer        0
Model               0
Prod. year          0
Category            0
Leather interior    0
Fuel type           0
Engine volume       0
Mileage             0
Cylinders           0
Gear box type       0
Drive wheels        0
Doors               0
Wheel               0
Color               0
Airbags             0
dtype: int64

In [253]:
#checking for duplicate values
train_df.duplicated().sum()

313

In [254]:
#drop duplicate values
train_df = train_df.drop_duplicates(keep='first')

In [255]:
#checking for duplicate values again
train_df.duplicated().sum() #is fine now

0

In [256]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,18924.0,4.557538e+07,937546.823889,20746880.0,45695007.5,45771914.5,45801742.25,45816654.0
Price,18924.0,1.858744e+04,192135.630726,1.0,5331.0,13172.0,22063.00,26307500.0
Prod. year,18924.0,2.010914e+03,5.665749,1939.0,2009.0,2012.0,2015.00,2020.0
Cylinders,18924.0,4.580216e+00,1.200223,1.0,4.0,4.0,4.00,16.0
Airbags,18924.0,6.568379e+00,4.322323,0.0,4.0,6.0,12.00,16.0


# Data Preperation

In [257]:
train_df.columns = [i.lower() for i in train_df.columns]
i

['price',
 'levy',
 'manufacturer',
 'model',
 'prod. year',
 'category',
 'leather interior',
 'fuel type',
 'engine volume',
 'mileage',
 'cylinders',
 'gear box type',
 'drive wheels',
 'doors',
 'wheel',
 'color',
 'airbags']

In [258]:
train_df.columns

Index(['id', 'price', 'levy', 'manufacturer', 'model', 'prod. year',
       'category', 'leather interior', 'fuel type', 'engine volume', 'mileage',
       'cylinders', 'gear box type', 'drive wheels', 'doors', 'wheel', 'color',
       'airbags'],
      dtype='object')

In [259]:
#drop ID
train_df.drop('id',axis=1, inplace= True)

In [260]:
df = train_df.copy()

In [261]:
#value_counts for df['levy']
df['levy'].value_counts()

-       5709
765      482
891      453
639      403
640      398
        ... 
3156       1
2908       1
1279       1
1719       1
1901       1
Name: levy, Length: 559, dtype: int64

In [262]:
#let's change '-' with the '765' which is the biggest second value
df['levy'] = df['levy'].str.replace('-', '765')

In [263]:
#change the dtype of df['levy']
df['levy'] = df['levy'].astype('int64')

In [264]:
#change df['mileage'] to the numbers by deleting km's
re.findall(r'\d+','168966 km')

['168966']

In [265]:
df['mileage'] = df['mileage'].str.replace(('\D+'),' ')

In [266]:
df['mileage'] = df['mileage'].astype('int64')

In [267]:
re.findall(r'\S+','2.0 Turbo')[0]

'2.0'

In [268]:
df['engine volume'] = df['engine volume'].str.extract('(\S+)')

In [269]:
df['engine volume'] = df['engine volume'].astype('float')

In [270]:
#looking for unoque values in df['doors']
df['doors'].unique()

array(['04-May', '02-Mar', '>5'], dtype=object)

In [271]:
df['doors'] = df['doors'].replace({'04-May':'4', '02-Mar':'2', '>5':'5'})

In [272]:
df['cylinders'] = df['cylinders'].astype('int64')

In [273]:
df['doors'] = df['doors'].astype('int64')

In [274]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18924 entries, 0 to 19236
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             18924 non-null  int64  
 1   levy              18924 non-null  int64  
 2   manufacturer      18924 non-null  object 
 3   model             18924 non-null  object 
 4   prod. year        18924 non-null  int64  
 5   category          18924 non-null  object 
 6   leather interior  18924 non-null  object 
 7   fuel type         18924 non-null  object 
 8   engine volume     18924 non-null  float64
 9   mileage           18924 non-null  int64  
 10  cylinders         18924 non-null  int64  
 11  gear box type     18924 non-null  object 
 12  drive wheels      18924 non-null  object 
 13  doors             18924 non-null  int64  
 14  wheel             18924 non-null  object 
 15  color             18924 non-null  object 
 16  airbags           18924 non-null  int64 

In [275]:
df.head()

,price,levy,manufacturer,model,prod. year,category,leather interior,fuel type,engine volume,mileage,cylinders,gear box type,drive wheels,doors,wheel,color,airbags
0,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005,6,Automatic,4x4,4,Left wheel,Silver,12
1,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3.0,192000,6,Tiptronic,4x4,4,Left wheel,Black,8
2,8467,765,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000,4,Variator,Front,4,Right-hand drive,Black,2
3,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966,4,Automatic,4x4,4,Left wheel,White,0
4,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901,4,Automatic,Front,4,Left wheel,Silver,4


# Encoding

In [276]:
#select categorical features
categoircal_features = list()
for col in df.columns:
  if df[col].dtype == 'object':
    categoircal_features.append(col)
categoircal_features

['manufacturer',
 'model',
 'category',
 'leather interior',
 'fuel type',
 'gear box type',
 'drive wheels',
 'wheel',
 'color']

In [277]:
#select numerical features
numerical_features = list()
for col in df.columns:
  if df[col].dtype != 'object':
    numerical_features.append(col)
numerical_features

['price',
 'levy',
 'prod. year',
 'engine volume',
 'mileage',
 'cylinders',
 'doors',
 'airbags']

In [278]:
#encoding with get dummies
final_df = pd.get_dummies(df,prefix= categoircal_features)

#print final dataframe
final_df

,price,levy,prod. year,engine volume,mileage,cylinders,doors,airbags,manufacturer_ACURA,manufacturer_ALFA ROMEO,...,color_Green,color_Grey,color_Orange,color_Pink,color_Purple,color_Red,color_Silver,color_Sky blue,color_White,color_Yellow
0,13328,1399,2010,3.5,186005,6,4,12,0,0,...,0,0,0,0,0,0,1,0,0,0
1,16621,1018,2011,3.0,192000,6,4,8,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8467,765,2006,1.3,200000,4,4,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3607,862,2011,2.5,168966,4,4,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,11726,446,2014,1.3,91901,4,4,4,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,8467,765,1999,2.0,300000,4,2,5,0,0,...,0,0,0,0,0,0,1,0,0,0
19233,15681,831,2011,2.4,161600,4,4,8,0,0,...,0,0,0,0,0,1,0,0,0,0
19234,26108,836,2010,2.0,116365,4,4,4,0,0,...,0,1,0,0,0,0,0,0,0,0
19235,5331,1288,2007,2.0,51258,4,4,4,0,0,...,0,0,0,0,0,0,0,0,0,0


#Modelling

In [279]:
#setting X and y
y = final_df['price']
X = final_df.drop('price',axis=1)
#train test split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size= 0.2, random_state= 42)

In [280]:
#scalling on X(all indepentedn variables)
rs = RobustScaler()
X = rs.fit_transform(X)

In [296]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X,y)

LinearRegression()

In [297]:
predictions = lr.predict(X_test)
r2_score(predictions,y_test)

-0.7900381762104287

In [298]:
predictions

array([8344402.68988369, 8312585.83418111, 8311760.83159564, ...,
       8279101.86111465, 8274385.15427993, 8120688.37121896])

In [299]:
y_test

3499     11604
9232       470
1810     20162
9625      7527
8522     18817
         ...  
15109    27654
1685      2509
11948      110
19008    16308
17096    47201
Name: price, Length: 3785, dtype: int64